In [2]:
# import os, sys
# from google.colab import drive
# drive.mount('/content/drive')
# my_path = '/content/notebooks'

In [5]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from konlpy.tag import Okt
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
#from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

okt = Okt()

In [6]:
test_data = pd.read_csv('test_data.csv' , encoding = 'euc-kr')
train_data = pd.read_csv('train_data.csv' , encoding = 'euc-kr')

In [7]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
import konlpy
print(konlpy.__version__)
print(pd.__version__)
print(np.__version__)

2.12.0
2.12.0
0.6.0
1.4.4
1.23.5


In [8]:
# loaded_model = load_model('/content/drive/MyDrive/data_review/best_model.h5')
# print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

# 버전 확인
# pip install --upgrade tensorflow==2.12.0
# pip install --upgrade keras==2.12.0

loaded_model = load_model('best_model2.h5')

In [9]:
# 불용어를 지정하여 필요없는 토큰들은 제거하도록 합니다.

stopwords = ['도', '는', '다', '의', '가', '이', '은', 
             '한', '에', '하', '고', '을', '를', '인', '듯', 
             '과', '와', '네', '들', '듯', '지', '임', '게']

In [10]:
# 단어 집합(vocabulary)의 크기 : 64152
# 등장 빈도가 1번 이하인 희귀 단어의 수: 29480
# 단어 집합에서 희귀 단어의 비율: 45.953360768175585
# 전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.289171484296422

In [11]:
# total_cnt = 64152
# rare_cnt = 29480
# vocab_size = total_cnt - rare_cnt + 2
# tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 

In [12]:
# 긍정 리뷰보다는 부정 리뷰가 좀 더 길게 작성된 경향이 있는 것 같습니다.

X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [13]:
# 정수 인코딩

# 기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행해야 합니다. 
# 훈련 데이터에 대해서 단어 집합(vocaburary)을 만들어봅시다.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [14]:
total_cnt = 64152
rare_cnt = 29480
vocab_size = total_cnt - rare_cnt + 2

In [15]:
# 단어 집합의 크기는 34674개입니다. 
# 이를 토크나이저의 인자로 넘겨주고, 텍스트 시퀀스를 정수 시퀀스로 변환합니다. 
# 정수 인코딩 과정에서 이보다 큰 숫자가 부여된 단어들은 OOV로 변환하겠습니다.

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [16]:
max_len = 330

In [17]:
# 훈련용 리뷰의 99.99%가 330 이하의 길이를 가집니다. 훈련용 리뷰를 길이 330으로 패딩하겠습니다.

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)


In [18]:
# # test파일 업로드

# from google.colab import files

# # 파일 선택 대화상자 표시
# uploaded = files.upload()

# # 업로드한 파일 확인
# for filename in uploaded.keys():
#     print(f'Uploaded file: {filename}')


In [22]:
# test 파일 업로드 2
#uploaded
# test_re = pd.read_csv('/content/drive/MyDrive/(주)시더스초밥제주연동점_review.csv' , encoding = 'euc-kr')
test_re = pd.read_csv('(주)시더스초밥제주연동점_review.csv' , encoding = 'euc-kr')
test_re = pd.read_excel('(주)서촌제 스시100_review.xlsx')
test_re = pd.read_csv('캐글학습용데이터kr3.tsv', delimiter='\t')

In [23]:
test_re

,Rating,Review
0,1,숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1,1,고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
2,1,"잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어..."
3,1,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
4,1,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...
...,...,...
641757,2,"요즘, 핫하게,,, 떠오르고 있는 중국집. , 맥주의 여 파루 속이 안 좋지만 와봄..."
641758,0,원래 글 안 쓰는데 이거는 정말 다른 분들 위해서 써야 할 것 같네요 방금 포장 주...
641759,1,"우리 팀 단골집, 술 먹고 다음 날 가면 푸짐하게 배불리 해장할 수 있는 곳, 주말..."
641760,2,"원래는 평택에 있었는데, 연남동에도 최근에 생겨서 방문했는데.. 진짜 줄이 어마어마..."


In [24]:
# 결측치 처리
test_re = test_re.dropna().reset_index(drop=True)

In [25]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen = max_len)

    score = float(loaded_model.predict(pad_new))
    if (score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else :
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))


In [26]:
# test
for num , i in enumerate(test_re.Review[:30].tolist()) :
    print(num)
    print(i)
    print(sentiment_predict(i))
    print()

0
숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편이고 살짝 레트로 감성으로 분위기 잡아놨습니다. 모든 직원분들께서 전부 가능하다고 멘트 쳐주시며, 고기는 초반 커팅까지는 구워주십니다. 가격 저렴한 편 아니지만 맛은 준수합니다. 등심덧살이 인상 깊었는데 구이로 별로일 줄 알았는데 육향 짙고 얇게 저며 뻑뻑하지 않았습니다. 하이라이트는 된장찌개. 진짜 굿입니다. 버터 간장밥, 골뱅이 국수 등 나중에 더 맛봐야 할 것들은 남겨뒀습니다.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 481ms/st